In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from passivbot_futures import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import PrettyPrinter
from common_functions import sort_dict_keys

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)
pp = PrettyPrinter()

In [ ]:
user = 'your_user_name'
settings = load_settings(user)
s = settings['symbol']

In [ ]:
try:
    await bot.cc.close()
except:
    pass
bot = await create_bot(user, settings)
cc = bot.cc

In [ ]:
n_days_load = 7
adf = await bot.load_trades(s, n_days_load)

In [ ]:
adf.price.iloc[::100].plot()

In [ ]:
# modify settings
settings['entry_amount'] = 0.001
settings['leverage'] = 100
settings['markup'] = 0.0014
settings['flashcrash_factor'] = 0.001
settings

In [ ]:
logs, trades, adf = backtest(adf, settings)

tdf = pd.DataFrame(trades).set_index('agg_id')
adf.price.plot()
bdf = tdf[tdf.side == 'buy']
sdf = tdf[tdf.side == 'sel']
bdf.price.plot(style='bo')
sdf.price.plot(style='ro')
tdf

In [ ]:
ldf = pd.DataFrame(logs)
realized_pnl_sum = ldf.realized_pnl_sum.max()
print(realized_pnl_sum)
ldf

In [ ]:
# n double downs
(ldf.pos_amount.abs().sort_values().tail(20) * 1000).apply(np.log2)

In [ ]:
max_needed_margin = ldf.initial_margin_max.max()
max_used_margin = ldf.initial_margin.max()
print('max_needed_margin', max_needed_margin)
print('ldf.pos_amount.min()', ldf.pos_amount.min())
print('ldf.pos_amount.max()', ldf.pos_amount.max())
print('np.log2(ldf.pos_amount.abs().max() * 1000)', np.log2(ldf.pos_amount.abs().max() * 1000))

In [ ]:
ldf.realized_pnl_sum.dropna().plot()

In [ ]:
ldf.pos_amount.plot()

In [ ]:
gain = (realized_pnl_sum + max_needed_margin) / max_needed_margin
n_days = (tdf.timestamp.max() - tdf.timestamp.min()) / 1000 / 60 / 60 / 24
adg = gain ** (1 / n_days)
average_daily_abs_profit = realized_pnl_sum / n_days
print('max_used_margin', max_used_margin)
print('max_needed_margin', max_needed_margin)
print('realized_pnl_sum', realized_pnl_sum)
print('gain', gain)
print('n_days', n_days)
print('adg', adg)
print('average_daily_abs_profit', average_daily_abs_profit)